# VSSo core synthetic data generator
This notebook is used to quickly generate a `Vehicle` with static and dynamic propeties using the Vehicle Signal Specification Ontology (VSSo).
In VSSo, an `StaticVehicleProperty` refers to the static information about the vehicle (e.g., `VehicleIdentificationBrand`, `VehicleIdentificationModel`, etc.).
Whereas a `DynamcVehicleProperty` refers to the actual signal of interest whose data values are dynamic over time (e.g., `Speed`).
In the case of `DynamcVehicleProperty`, VSSo distinguishes two sub-classes: `ObservableVehicleProperty`, where data values are read-only; and `ActuatableVehicleProperty`, whose data values can be read and written.

The synthetic data that this notebook creates is NOT a time-series, as it only contains **one value for each property**.
This is enough to demonstrate the funtionality of the **VSSo core** model.
Other use-cases and demonstrations will be added accordingly to the same `vsso-demo` repository when they become available.

## Instructions
Almost all values are generated randomly within a range that is appropiate to each property.
If other values or more properties are desired, simply modify the corresponding line.

You can run this notebook using [Jupyter](https://jupyter.org/install).
Make sure that you are using a kernel that has already intalled the `environment.yml` file available in the working directory.
If you use `conda`, do the following in the working directory:
```zsh
% conda env create --name vsso-demo --file environment.yml
% conda activate vsso-demo
% jupyter lab
```

Alternativaly, you can use [Binder](https://mybinder.org) to run the notebook online.

To create a new `vsso_core_data_example.ttl`, just run all cells sequentially.
> In a Jupyter noteboook do: `Kernel`>`Restart Kernel and Run All Cells...`

Once the `vsso_core_data_example.ttl` file is created, you can test the queries available in the `vsso_core_query_examples.md` file with a `SPARQL` end point of your preference.
> Make sure that you also import the VSSo ontology.

### Dependencies

In [1]:
import random
import string
from rdflib import URIRef, Literal, Namespace, RDF, Graph, XSD

### RDF graph configuration

In [2]:
num_vehicles = 1000  # how many vehicles you want to simulate?

In [3]:
rdf_graph = Graph()  # create an empty RDF graph

vsso = Namespace("https://www.w3.org/vsso/")  # default root URI of the VSSo ontology
indv = Namespace("https://www.w3.org/vsso/example/")  # root URI used for the individuals

# Bind the prefixes for the abbreviated serialization
rdf_graph.bind("vsso", "https://www.w3.org/vsso/")
rdf_graph.bind("indv", "https://www.w3.org/vsso/example/")

In [4]:
def add_static_properties(graph, vehicle_identification_number, vehicle_indvividual, special_properties):
    '''Adds the static properties to the given RDF graph.'''
    
    door_count = 2 if (special_properties['seat_row_count'] == 1) else 4
    
    static_vehicle_properties = [
        {'relationship_uri':vsso.hasVehicleIdentificationVIN, 'value':vehicle_identification_number},
        {'relationship_uri':vsso.hasVehicleIdentificationBrand, 'value':random.choice(string.ascii_uppercase)},
        {'relationship_uri':vsso.hasVehicleIdentificationModel, 'value':random.choice(string.ascii_lowercase)},
        {'relationship_uri':vsso.hasVehicleIdentificationYear, 'value':random.choice(list(range(2015,2022)))},
        {'relationship_uri':vsso.hasChassisAxleCount, 'value':random.choice([2, 3, 4, 5, 6])},
        {'relationship_uri':vsso.hasChassisCurbWeight, 'value':random.randint(1000, 2000)},
        {'relationship_uri':vsso.hasChassisGrossWeight, 'value':random.randint(3000, 5000)},
        {'relationship_uri':vsso.hasChassisHeight, 'value':random.randint(1500, 3500)},
        {'relationship_uri':vsso.hasChassisLength, 'value':random.randint(2000, 4000)},
        {'relationship_uri':vsso.hasChassisTowWeight, 'value':random.randint(2000, 4000)},
        {'relationship_uri':vsso.hasChassisTrack, 'value':random.randint(1000, 2000)},
        {'relationship_uri':vsso.hasChassisWheelbase, 'value':random.randint(2000, 3000)},
        {'relationship_uri':vsso.hasChassisWidth, 'value':random.randint(2000, 3000)},
        {'relationship_uri':vsso.hasTransmissionType, 'value':random.choice(["sequential", "H", "automatic", "DSG", "CVT"])},
        {'relationship_uri':vsso.hasCombustionEngineConfiguration, 'value':random.choice(["straight", "V", "boxer", "W", "rotary", "radial", "square", "H", "U", "opposed", "X"])},
        {'relationship_uri':vsso.hasCombustionEngineDisplacement, 'value':random.randint(1000, 6000)},
        {'relationship_uri':vsso.hasCombustionEngineFuelType, 'value':special_properties['fuel_type']},
        {'relationship_uri':vsso.hasFuelSystemFuelType, 'value':special_properties['fuel_type']},
        {'relationship_uri':vsso.hasCombustionEngineMaxPower, 'value':random.randint(80, 400)},
        {'relationship_uri':vsso.hasCombustionEngineMaxTorque, 'value':random.randint(100, 10000)},
        {'relationship_uri':vsso.hasCabinSeatRowCount, 'value':special_properties['seat_row_count']},
        {'relationship_uri':vsso.hasVehicleIdentificationvehicleSeatingCapacity, 'value':random.randint(2, 7)},
        {'relationship_uri':vsso.hasCabinDoorCount, 'value':door_count},
        {'relationship_uri':vsso.hasSteeringWheelPosition, 'value':random.choice(["left", "right"])}, 
    ]
    
    for i in static_vehicle_properties:
        graph.add((vehicle_indvividual, i['relationship_uri'] , Literal(i['value'])))

In [5]:
def add_dynamic_properties(rdf_graph, vehicle_identification_number, vehicle_indvividual, special_properties):
    '''Adds the dynamic properties to the given RDF graph.'''
   
    # Dynamic properties with one instance
    
    dynamic_vehicle_properties = [
        {'name':'AccelerationLongitudinal', 'value':random.randint(-10, 10)},
        {'name':'AmbientAirTemperature', 'value':random.randint(0, 40)},
        {'name':'TravelledDistance', 'value':random.randint(0, 1000000)},
        {'name':'Speed', 'value':random.randint(0, 200)},
        {'name':'TransmissionGear', 'value':random.choice([-1, 1, 2, 3, 4, 5, 6, 7, 8])},
        {'name':'SteeringWheelAngle', 'value':random.randint(-540, 540)},
    ]

    # Dynamic properties with multiple instances
    
    # Create window instances, equal to the number of doors
    for row in range(1, special_properties['seat_row_count']+1):
        for side in ["Left", "Right"]:
            instance = f'Row{row}-{side}'
            dynamic_vehicle_properties.append({'name':f'WindowisOpen-{instance}', 
                                               'value':random.choice([True, False]), 
                                               'instances':[f'Row{row}',f'{side}']}) 
            
    for i in dynamic_vehicle_properties:
        name = i['name']
        individual_uri = getattr(indv, f'{vehicle_identification_number}-{name}')
        class_uri = getattr(vsso, f'{name}')

        rdf_graph.add((vehicle_indvividual, vsso.hasDynamicVehicleProperty, individual_uri))
        rdf_graph.add((individual_uri, RDF.type, class_uri))
        rdf_graph.add((individual_uri, vsso.holdsState, Literal(i['value'])))
    
    if 'instances' in i:
        for instance in i['instances']:
            rdf_graph.add((individual_uri, vsso.hasInstance, Literal(instance)))

In [6]:
for vehicle_identification_number in range(num_vehicles):
    
    # Add in this dict properties that require special considerations (i.e., used multiple times)
    special_properties = {
        'fuel_type': random.choice(["gasoline", "diesel", "E85", "CNG"]),  
        'seat_row_count': random.randint(1, 3)
    }

    # Create Vehicle individual using the VIN
    vehicle_indvividual = getattr(indv, f'{vehicle_identification_number}-Vehicle')
    
    add_static_properties(rdf_graph, vehicle_identification_number, vehicle_indvividual, special_properties)
    add_dynamic_properties(rdf_graph, vehicle_identification_number, vehicle_indvividual, special_properties)

In [7]:
rdf_graph.serialize("vsso_core_data_example.ttl", format='ttl')